---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [30]:
trainData = pd.read_csv('train.csv',encoding = 'ISO-8859-1')
testData = pd.read_csv('test.csv')
address = pd.read_csv('addresses.csv')
latlons = pd.read_csv('latlons.csv')

In [13]:
print(trainData.head())
print(address.head())
print(latlons.head())

   ticket_id                                     agency_name  \
0      22056  Buildings, Safety Engineering & Env Department   
1      27586  Buildings, Safety Engineering & Env Department   
2      22062  Buildings, Safety Engineering & Env Department   
3      22084  Buildings, Safety Engineering & Env Department   
4      22093  Buildings, Safety Engineering & Env Department   

     inspector_name                      violator_name  \
0   Sims, Martinzie  INVESTMENT INC., MIDWEST MORTGAGE   
1  Williams, Darrin           Michigan, Covenant House   
2   Sims, Martinzie                    SANDERS, DERRON   
3   Sims, Martinzie                       MOROSI, MIKE   
4   Sims, Martinzie                    NATHANIEL, NEAL   

   violation_street_number violation_street_name  violation_zip_code  \
0                   2900.0                 TYLER                 NaN   
1                   4311.0               CENTRAL                 NaN   
2                   1449.0            LONGFELLOW  

In [31]:
address = address.set_index('address').join(latlons.set_index('address'),how='left')
address.head()

,ticket_id,lat,lon
address,,,
"-11064 gratiot, Detroit MI",328722,42.406935,-82.995599
"-11871 wilfred, Detroit MI",350971,42.411288,-82.993674
"-15126 harper, Detroit MI",344821,42.406402,-82.957525
"0 10th st, Detroit MI",24928,42.325689,-83.064330
"0 10th st, Detroit MI",71887,42.325689,-83.064330


In [32]:
trainData = trainData.set_index('ticket_id').join(address.set_index('ticket_id'))
testData = testData.set_index('ticket_id').join(address.set_index('ticket_id'))
#trainData.head()

In [ ]:
#print(trainData['agency_name'].unique())
print(len(trainData['agency_name'].unique()))
print(trainData.shape)
#print(trainData['inspector_name'].unique())
print(len(trainData['inspector_name'].unique()))
trainData.columns

In [33]:
trainData=trainData.dropna(subset=['compliance'])
y_train = trainData['compliance']
trainData.hearing_date.fillna(method='pad', inplace=True)
testData.hearing_date.fillna(method='pad', inplace=True)

In [ ]:
trainData.columns

In [34]:
list_of_col_to_remove_train = ['payment_amount', 'payment_date','payment_status',
                              'balance_due','collection_status', 'compliance_detail',
                              'compliance']
trainData.drop(list_of_col_to_remove_train, axis=1, inplace=True)

In [8]:
print(trainData.shape)
testData.shape

(159880, 28)


(61001, 28)

In [35]:
trainData['TotalFee'] = trainData['admin_fee'] + trainData['state_fee']+\
trainData['late_fee'] - trainData['discount_amount'] + trainData['clean_up_cost']+\
trainData['judgment_amount']

testData['TotalFee'] = testData['admin_fee'] + testData['state_fee']+\
testData['late_fee'] - testData['discount_amount'] + testData['clean_up_cost']+\
testData['judgment_amount']

In [36]:
trainData['gapdays'] = (pd.to_datetime(trainData['hearing_date'])-pd.to_datetime(trainData['ticket_issued_date'])).dt.days
testData['gapdays'] = (pd.to_datetime(testData['hearing_date'])-pd.to_datetime(testData['ticket_issued_date'])).dt.days


In [11]:
list_of_col_to_remove_all = ['violation_street_number', 'violation_street_name',
                             'violation_zip_code','mailing_address_str_number',
                              'mailing_address_str_name','city',
                            'state','zip_code', 'non_us_str_code', 'country',
                            'violation_description','disposition','violator_name',
                            'grafitti_status','ticket_issued_date','hearing_date',
                            'admin_fee', 'state_fee', 'late_fee','discount_amount',
                             'clean_up_cost', 'judgment_amount','inspector_name']

In [12]:
trainData.drop(list_of_col_to_remove_all, axis=1, inplace=True)
testData.drop(list_of_col_to_remove_all, axis=1, inplace=True)

In [13]:
print(trainData.shape)
print(testData.shape)

(159880, 7)
(61001, 7)


In [24]:
print(trainData.columns)
testData.columns
#trainData.head()

Index(['agency_name', 'violation_code', 'fine_amount', 'lat', 'lon',
       'TotalFee', 'gapdays'],
      dtype='object')


Index(['agency_name', 'violation_code', 'fine_amount', 'lat', 'lon',
       'TotalFee', 'gapdays'],
      dtype='object')

In [14]:
print(trainData.count())
print(testData.count())

trainData.lat.fillna(method='pad', inplace=True)
trainData.lon.fillna(method='pad', inplace=True)

testData.lat.fillna(method='pad', inplace=True)
testData.lon.fillna(method='pad', inplace=True)


agency_name       159880
violation_code    159880
fine_amount       159880
lat               159878
lon               159878
TotalFee          159880
gapdays           159880
dtype: int64
agency_name       61001
violation_code    61001
fine_amount       61001
lat               60996
lon               60996
TotalFee          61001
gapdays           61001
dtype: int64


In [15]:
DataT = pd.concat([trainData,testData],axis=0)
DataT.shape

feature_to_be_splitted = ['violation_code', 'agency_name']
DataT = pd.get_dummies(DataT, columns=feature_to_be_splitted)

type(DataT)
DataT.shape

train_data = DataT.iloc[0:trainData.shape[0],]
print(train_data.shape)
test_data = DataT.iloc[trainData.shape[0]+1:,]
test_data.shape

(159880, 233)


(61000, 233)

In [ ]:
len(trainData['violation_code'].unique())

In [16]:
lgr = LogisticRegression().fit(train_data, y_train)

In [17]:
nb = GaussianNB().fit(train_data, y_train)

In [18]:
rf = RandomForestClassifier().fit(train_data, y_train)

In [19]:
gb = GradientBoostingClassifier().fit(train_data, y_train)

In [20]:
nn = MLPClassifier(solver='lbfgs', random_state = 0).fit(train_data, y_train)

In [21]:
dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(train_data, y_train)

In [ ]:
svec = SVC(kernel='linear', C=1).fit(train_data,y_train)

In [35]:
dt = DecisionTreeClassifier(max_depth=2).fit(train_data, y_train)

In [34]:
test_proba = clf.predict_proba(test_data)[:,1]

/home/pegah/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [22]:
pred_lgr = lgr.predict(test_data)
pred_nb = nb.predict(test_data)
pred_rf = rf.predict(test_data)
pred_gb = gb.predict(test_data)
pred_nn = nn.predict(test_data)
pred_dummy = dummy_majority.predict(test_data)
#pred_svec = svec.predict(test_data)
#pred_dt = dt.predict(test_data)

In [26]:
# accuracy is the default scoring metric
print('Cross-validation (accuracy)', cross_val_score(gb, train_data, y_train, cv=5))
# use AUC as scoring metric
print('Cross-validation (AUC)', cross_val_score(gb, train_data, y_train, cv=5, scoring = 'roc_auc'))

('Cross-validation (accuracy)', array([ 0.9293242 ,  0.93858085,  0.94054916,  0.92813135,  0.93413604]))
('Cross-validation (AUC)', array([ 0.76508695,  0.75045503,  0.80743313,  0.75879914,  0.75067282]))


In [27]:
# accuracy is the default scoring metric
print('Cross-validation (accuracy)', cross_val_score(lgr, train_data, y_train, cv=5))
# use AUC as scoring metric
print('Cross-validation (AUC)', cross_val_score(lgr, train_data, y_train, cv=5, scoring = 'roc_auc'))
np.mean(cross_val_score(lgr, train_data, y_train, cv=5))

('Cross-validation (accuracy)', array([ 0.92719767,  0.92772931,  0.9276645 ,  0.92759969,  0.92631744]))
('Cross-validation (AUC)', array([ 0.73149521,  0.73869091,  0.80574285,  0.75974651,  0.76750552]))


0.92730171997838373

In [ ]:
# accuracy is the default scoring metric
print('Cross-validation (accuracy)', cross_val_score(rf, train_data, y_train, cv=5))
# use AUC as scoring metric
print('Cross-validation (AUC)', cross_val_score(rf, train_data, y_train, cv=5, scoring = 'roc_auc'))

In [22]:
print(sum(pred_lgr))
print(sum(pred_nb))
print(sum(pred_rf))
print(sum(pred_gb))
print(sum(pred_nn))
print(sum(pred_dummy))
print(sum(pred_svec))

586.0
13359.0
2292.0
1423.0
1143.0
0.0


In [29]:
# add regularization 
rocVec = np.zeros(5)
for iter,g in enumerate([0.01, 0.1, 1, 10, 100]):
    lgr = LogisticRegression(C=g).fit(train_data, y_train)
    rocVec[iter] = np.mean(cross_val_score(lgr, train_data, y_train, cv=5, scoring = 'roc_auc'))
print(rocVec)

[ 0.76362913  0.76019965  0.7606362   0.76062513  0.7605634 ]


In [ ]:
#blight_model()